In [1]:
from pyspark.context import SparkContext
from pyspark.context import SparkConf

context = SparkContext.getOrCreate(SparkConf().setMaster("local"))
rdd_lines = context.textFile("/home/jovyan/Thunderbird-100k.csv")

21/09/22 13:52:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/09/22 13:52:32 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
import datetime

rdd_alerts = rdd_lines.map(lambda line: (line.strip().split()[2], True) if line.startswith("-") else (line.strip().split()[2], False)) \
            .map(lambda line: (datetime.datetime.strptime(line[0],"%Y.%m.%d").weekday(), line[1]))

rdd_alerts.take(1)

[(2, True)]

In [3]:
non_alerts = rdd_alerts.filter(lambda tup: tup[1]).map(lambda tup: (tup[0], 1)).reduceByKey(lambda a, b: a + b).collect()
alerts = rdd_alerts.filter(lambda tup: not tup[1]).map(lambda tup: (tup[0], 1)).reduceByKey(lambda a, b: a + b).collect()

print("Alerts:")
for word in alerts:
    print(word)
print("Non alerts:")
for word in non_alerts:
    print(word)


Alerts:
(1, 37)
Non alerts:
(2, 50000)
(1, 49963)


In [8]:
import re

def to_list(a):
    return [a]

def append(a, b):
    a.append(b)
    return a

def extend(a, b):
    a.extend(b)
    return a

result_list = (rdd_lines \
    .filter(lambda line: len(line.strip().split(":", 3)) == 4) \
    # map to (year, [word1, word2, ...])
    .map(lambda line: (datetime.datetime.strptime(line.strip().split()[2],"%Y.%m.%d").year, line.strip().split(":",3)[3].split())) \
    # flatmap to (year, word1) (year, word2), ...
    .flatMap(lambda tup: [(tup[0], word) for word in tup[1]]) \
    .map(lambda tup: (tup[0], re.sub("[^a-zA-Z]+", "", tup[1]).lower().strip())) \
    .filter(lambda tup: len(tup[1]) > 0) \
     # maps to ((year, word), 1)
    .map(lambda tup: ((tup[0], tup[1]), 1)) \
    # counts per (year, word)
    .reduceByKey(lambda counts1, counts2: counts1 + counts2) \
    # maps to ((year, counts), word)
    .map(lambda tup: ((tup[0][0], tup[1]),tup[0][1])) \
    # sorts by (year, counts)
    .sortByKey(ascending=False)
    # maps to (year, (word, counts))
    .map(lambda tup: (tup[0][0], (tup[1], tup[0][1]))) \
    # get (year, [(word1, count1), (word2, count2), ...])
    .combineByKey(to_list, append, extend) \
    # limit size of words lists to 50
    .map(lambda tup: (tup[0], (tup[1][0:49]))))

for year_words in result_list.collect():
    print(year_words[0])
    for word_counts in year_words[1]:
        print(word_counts)



2006
('connection', 13425)
('from', 7728)
('closed', 6825)
('to', 6317)
('user', 4534)
('on', 4275)
('s', 3965)
('synchronized', 3835)
('stratum', 3831)
('lost', 3410)
('now', 3387)
('local', 3346)
('authenticated', 3253)
('running', 3245)
('privileges', 3224)
('coming', 3221)
('disconnected', 3214)
('tbird', 3155)
('for', 2844)
('acpi', 2835)
('pamkrb', 2671)
('pciehp', 2352)
('no', 2072)
('pci', 1878)
('at', 1675)
('ibsmsweepc', 1630)
('oshp', 1584)
('root', 1529)
('lustreerror', 1459)
('succeeded', 1356)
('failsx', 1328)
('interrupt', 1318)
('command', 1265)
('configuration', 1231)
('session', 1210)
('device', 1198)
('irq', 1120)
('change', 1070)
('eth', 1007)
('clientcptlrpcexpireonerequest', 998)
('error', 955)
('via', 937)
('timeout', 899)
('new', 884)
('driver', 882)
('by', 859)
('not', 852)
('using', 846)
('flag', 836)
2005
('from', 12931)
('not', 11525)
('got', 11331)
('any', 11291)
('datathread', 11288)
('answer', 11288)
('datasource', 11288)
('an', 9433)
('cn', 8301)
('dn', 